In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import keras
import seq2seq
from seq2seq.models import AttentionSeq2Seq
from seq2seq.models import Seq2Seq
from keras.utils import multi_gpu_model
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import shutil
import pickle
import tqdm
from tqdm import tqdm

import os, sys
import numpy as np
from random import shuffle
from math import log, floor
import pandas as pd
import tensorflow as tf
import tensorboard as tb
from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
from keras import *
from keras.engine.topology import *
from keras.optimizers import *
import gensim
from gensim.models.word2vec import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *
from keras.utils import *
from sklearn.model_selection import *
import random
from random import shuffle
import re
# from collections import Counter
from keras.utils.generic_utils import *
from keras import regularizers
import string
import unicodedata as udata
import pickle
from keras.applications import *
from keras.preprocessing.image import *
from multiprocessing import *

Using TensorFlow backend.


In [7]:
with open('data/aries_simplified_profile_1002/8029408556a51064b21ef7a1cb21debb7c99210adfae0c5486b103b31b9bc44e_3304.trace.hooklog') as f:
    lines = f.read().splitlines()
print(lines)
for line in lines:
#     kk = line.replace("#|@|\|*|{|}|.|;|:|#RET"," ")
    kk = re.sub(r"[,.;@#?!&$:{}\\*%-]+\ *"," ",line)
    kk = kk.replace('PR','')
    kk = kk.replace('Ret','')
    kk = kk.replace('SUBK','')
    kk = kk.split(" ")
    kk = list(filter(None, kk))
#     kk = [x for x in kk if x]
    print(kk)
# kk

['LoadLibrary#PR@ARB@windowsshell@MANIFEST#Ret#N', 'CreateFile#PR@ARB@MANIFEST#PR@GENERIC_READ#PR@OPEN_EXISTING#PR@FILE_SHARE_DELETE;FILE_SHARE_READ#Ret#P', 'RegQueryValue#PR@HKCU@desktop\\#PR@SUBK@smoothscroll#PR@0#PR@77462a48#Ret#P', 'RegQueryValue#PR@HKCU@soft_ms_win_explorer\\advanced#PR@SUBK@enableballoontips#PR@0#PR@77462170#Ret#P', 'RegEnumValue#PR@HKLM@soft_ms_winNT_languagepack\\*#PR@SUBK@surr#PR@REG_DWORD#PR@2#Ret#0', 'RegQueryValue#PR@HKLM@sys_setup\\#PR@SUBK@systemsetupinprogress#PR@REG_DWORD#PR@0#Ret#0', 'RegQueryValue#PR@HKLM@sys_curCtlSet_ctl_sessionManager\\*#PR@SUBK@criticalsectiontimeout#PR@0#PR@7f9b0#Ret#0', 'RegQueryValue#PR@HKLM@soft_ms_ole\\*#PR@SUBK@rwlockresourcetimeout#PR@0#PR@7f9b4#Ret#P', 'RegQueryValue#PR@HKLM@sys_curCtlSet_ctl_mediaProperties\\*#PR@SUBK@wheel#PR@0#PR@76b60b9c#Ret#0', 'RegCreateKey#PR@SUBK@6c#PR@Software\\Microsoft\\Multimedia\\Audio#Ret#0', 'RegQueryValue#PR@SUBK@soft_ms_multimedia\\audio#PR@SUBK@systemformats#PR@0#PR@0#Ret#0', 'RegQueryVal

In [11]:
' '.join(['LoadLibrary', 'ARB', 'windowsshell', 'MANIFEST', 'N'])

'LoadLibrary ARB windowsshell MANIFEST N'

1. 列出各個term在整個corpus上的出現次數
2. 找出main api term出現最少的是誰，作為lower bound
3. 若parameter value出現的次數超過lower bound的話就拿掉該token(term)<=太common的不要(要記錄下來哪些不要)

- 缺點: 可能只會剩下罕見的東西

- 太罕見的不要。或是進行轉換

計算所有term
(整個corpus的)

In [75]:
corpus = []
dil= r"[,.;\-+^()/@#?!&$:{}\\*%~\'\"]+\ *" #等號、底線被保留
for hkl in tqdm(next(os.walk('data/aries_simplified_profile_1002/'))[2]):
    with open('data/aries_simplified_profile_1002/' + hkl , encoding='ISO 8859-1') as f:
        lines = f.read().splitlines()
    for line in lines:
        temp = re.sub(dil," ",line)
        temp = temp.replace('PR','')
        temp = temp.replace('Ret','')
        temp = temp.replace('SUBK','')
        temp = temp.split(" ")
        temp = [x for x in temp if x]
        for term in temp:
            if term not in corpus:
                corpus.append(term)
corpus_df = pd.DataFrame(pd.Series(sorted(corpus)),columns=['term'])
corpus_df['tf'] = 0

100%|██████████| 19987/19987 [06:57<00:00, 47.92it/s]


count by term's frequency

In [ ]:
for hkl in tqdm(next(os.walk('data/aries_simplified_profile_1002/'))[2]):
    with open('data/aries_simplified_profile_1002/' + hkl , encoding='ISO 8859-1') as f:
        lines = f.read().splitlines()
    for line in lines:
        temp = re.sub(dil," ",line)
        temp = temp.replace('PR','')
        temp = temp.replace('Ret','')
        temp = temp.replace('SUBK','')
        temp = temp.split(" ")
        temp = [x for x in temp if x]
        corpus_df.loc[corpus_df.term.isin(temp),'tf'] +=1
#         for term in temp:
#             corpus_df.loc[corpus_df['term']==term,'tf'] += 1

count by doc's frequency

In [98]:
# corpus_df = pd.DataFrame(pd.Series(sorted(corpus)),columns=['term'])
corpus_df['df'] = 0
for hkl in tqdm(next(os.walk('data/aries_simplified_profile_1002/'))[2]):
    with open('data/aries_simplified_profile_1002/' + hkl , encoding='ISO 8859-1') as f:
        lines = f.read().splitlines()
    line_word = []
    for line in lines:
        temp = re.sub(dil," ",line)
        temp = temp.replace('PR','')
        temp = temp.replace('Ret','')
        temp = temp.replace('SUBK','')
        temp = temp.split(" ")
        temp = [x for x in temp if x]
        line_word.extend(temp)
#         for term in temp:
#             if term not in line_word:
#                 line_word.append(term)
    line_word = set(line_word)
    corpus_df.loc[corpus_df.term.isin(line_word),'df'] +=1
#         for term in temp:
#             corpus_df.loc[corpus_df['term']==term,'tf'] += 1



  0%|          | 0/19987 [00:00<?, ?it/s]

  0%|          | 13/19987 [00:00<02:40, 124.51it/s]

  0%|          | 28/19987 [00:00<02:28, 134.62it/s]

  0%|          | 42/19987 [00:00<02:27, 134.91it/s]

  0%|          | 56/19987 [00:00<02:26, 135.77it/s]

  0%|          | 74/19987 [00:00<02:19, 142.73it/s]

  0%|          | 88/19987 [00:00<02:20, 141.64it/s]

  1%|          | 103/19987 [00:00<02:20, 141.41it/s]

  1%|          | 117/19987 [00:00<02:27, 134.56it/s]

  1%|          | 131/19987 [00:00<02:26, 135.16it/s]

  1%|          | 144/19987 [00:01<02:32, 129.94it/s]

  1%|          | 158/19987 [00:01<02:32, 130.42it/s]

  1%|          | 174/19987 [00:01<02:30, 131.73it/s]

  1%|          | 189/19987 [00:01<02:28, 133.00it/s]

  1%|          | 203/19987 [00:01<02:29, 132.49it/s]

  1%|          | 217/19987 [00:01<02:31, 130.57it/s]

  1%|          | 230/19987 [00:01<02:31, 130.43it/s]

  1%|          | 243/19987 [00:01<02:31, 130.01it/s]

  1%|▏         | 256/19987 [00:02<02:34, 12

 20%|██        | 4025/19987 [00:29<01:58, 135.15it/s]

 20%|██        | 4040/19987 [00:29<01:57, 135.17it/s]

 20%|██        | 4054/19987 [00:30<01:57, 135.11it/s]

 20%|██        | 4068/19987 [00:30<01:57, 135.02it/s]

 20%|██        | 4082/19987 [00:30<01:57, 135.01it/s]

 20%|██        | 4095/19987 [00:30<01:57, 134.96it/s]

 21%|██        | 4108/19987 [00:30<01:57, 134.69it/s]

 21%|██        | 4120/19987 [00:30<01:57, 134.64it/s]

 21%|██        | 4135/19987 [00:30<01:57, 134.66it/s]

 21%|██        | 4151/19987 [00:30<01:57, 134.73it/s]

 21%|██        | 4165/19987 [00:30<01:57, 134.65it/s]

 21%|██        | 4181/19987 [00:31<01:57, 134.71it/s]

 21%|██        | 4197/19987 [00:31<01:57, 134.78it/s]

 21%|██        | 4212/19987 [00:31<01:57, 134.77it/s]

 21%|██        | 4226/19987 [00:31<01:57, 134.69it/s]

 21%|██        | 4240/19987 [00:31<01:57, 134.59it/s]

 21%|██▏       | 4254/19987 [00:31<01:56, 134.60it/s]

 21%|██▏       | 4270/19987 [00:31<01:56, 134.66it/s]

 21%|██▏  

 40%|███▉      | 7986/19987 [00:59<01:30, 133.31it/s]

 40%|████      | 7999/19987 [01:00<01:29, 133.30it/s]

 40%|████      | 8011/19987 [01:00<01:29, 133.28it/s]

 40%|████      | 8023/19987 [01:00<01:29, 133.26it/s]

 40%|████      | 8035/19987 [01:00<01:30, 132.78it/s]

 40%|████      | 8049/19987 [01:00<01:29, 132.78it/s]

 40%|████      | 8060/19987 [01:00<01:29, 132.73it/s]

 40%|████      | 8073/19987 [01:00<01:29, 132.72it/s]

 40%|████      | 8085/19987 [01:00<01:29, 132.67it/s]

 41%|████      | 8098/19987 [01:01<01:29, 132.66it/s]

 41%|████      | 8110/19987 [01:01<01:29, 132.60it/s]

 41%|████      | 8122/19987 [01:01<01:29, 132.57it/s]

 41%|████      | 8138/19987 [01:01<01:29, 132.60it/s]

 41%|████      | 8154/19987 [01:01<01:29, 132.64it/s]

 41%|████      | 8168/19987 [01:01<01:29, 132.20it/s]

 41%|████      | 8182/19987 [01:01<01:29, 132.20it/s]

 41%|████      | 8194/19987 [01:02<01:29, 132.15it/s]

 41%|████      | 8206/19987 [01:02<01:29, 132.12it/s]

 41%|████ 

 59%|█████▉    | 11881/19987 [01:30<01:01, 131.68it/s]

 60%|█████▉    | 11898/19987 [01:30<01:01, 131.72it/s]

 60%|█████▉    | 11913/19987 [01:30<01:01, 131.74it/s]

 60%|█████▉    | 11929/19987 [01:30<01:01, 131.77it/s]

 60%|█████▉    | 11944/19987 [01:30<01:01, 131.77it/s]

 60%|█████▉    | 11959/19987 [01:30<01:00, 131.76it/s]

 60%|█████▉    | 11975/19987 [01:30<01:00, 131.79it/s]

 60%|██████    | 11993/19987 [01:30<01:00, 131.84it/s]

 60%|██████    | 12010/19987 [01:31<01:00, 131.88it/s]

 60%|██████    | 12026/19987 [01:31<01:00, 131.85it/s]

 60%|██████    | 12041/19987 [01:31<01:00, 131.85it/s]

 60%|██████    | 12056/19987 [01:31<01:00, 131.86it/s]

 60%|██████    | 12070/19987 [01:31<01:00, 131.67it/s]

 60%|██████    | 12087/19987 [01:31<00:59, 131.71it/s]

 61%|██████    | 12106/19987 [01:31<00:59, 131.77it/s]

 61%|██████    | 12123/19987 [01:31<00:59, 131.80it/s]

 61%|██████    | 12139/19987 [01:32<00:59, 131.83it/s]

 61%|██████    | 12155/19987 [01:32<00:59, 131.8

 79%|███████▉  | 15859/19987 [01:59<00:31, 132.22it/s]

 79%|███████▉  | 15876/19987 [02:00<00:31, 132.25it/s]

 80%|███████▉  | 15894/19987 [02:00<00:30, 132.29it/s]

 80%|███████▉  | 15912/19987 [02:00<00:30, 132.32it/s]

 80%|███████▉  | 15930/19987 [02:00<00:30, 132.34it/s]

 80%|███████▉  | 15947/19987 [02:00<00:30, 132.15it/s]

 80%|███████▉  | 15961/19987 [02:00<00:30, 132.13it/s]

 80%|███████▉  | 15976/19987 [02:00<00:30, 132.13it/s]

 80%|████████  | 15990/19987 [02:01<00:30, 132.11it/s]

 80%|████████  | 16003/19987 [02:01<00:30, 132.09it/s]

 80%|████████  | 16016/19987 [02:01<00:30, 132.07it/s]

 80%|████████  | 16031/19987 [02:01<00:29, 132.06it/s]

 80%|████████  | 16047/19987 [02:01<00:29, 132.08it/s]

 80%|████████  | 16061/19987 [02:01<00:29, 132.04it/s]

 80%|████████  | 16075/19987 [02:01<00:29, 132.05it/s]

 81%|████████  | 16090/19987 [02:01<00:29, 132.05it/s]

 81%|████████  | 16106/19987 [02:01<00:29, 132.08it/s]

 81%|████████  | 16121/19987 [02:02<00:29, 132.0

 99%|█████████▉| 19843/19987 [02:29<00:01, 132.34it/s]

 99%|█████████▉| 19859/19987 [02:30<00:00, 132.33it/s]

 99%|█████████▉| 19874/19987 [02:30<00:00, 132.33it/s]

100%|█████████▉| 19889/19987 [02:30<00:00, 132.32it/s]

100%|█████████▉| 19903/19987 [02:30<00:00, 132.32it/s]

100%|█████████▉| 19918/19987 [02:30<00:00, 132.33it/s]

100%|█████████▉| 19932/19987 [02:30<00:00, 132.31it/s]

100%|█████████▉| 19946/19987 [02:30<00:00, 132.32it/s]

100%|█████████▉| 19960/19987 [02:30<00:00, 132.32it/s]

100%|█████████▉| 19974/19987 [02:30<00:00, 132.32it/s]

100%|██████████| 19987/19987 [02:31<00:00, 132.32it/s]

filter out too common word and too less word by tf and df

In [109]:
#挑選原本32種main api，查看各自的上限跟下限作為過濾條件。太多出現的沒有鑑別力，太少出現的會使corpus數量太大所以轉換成UNK
corpus_df_ = corpus_df[corpus_df.df>116]
corpus_df_ = corpus_df_[corpus_df_.tf>134]
corpus_df_ = corpus_df_[corpus_df_.df<19066]
corpus_df_ = corpus_df_[corpus_df_.tf<1834644]
corpus_df_.reset_index(drop=True,inplace=True)
pickle.dump(file=open('output/asp1_tfdf_df_clean.pkl','wb'),obj=corpus_df_)
corpus_df_ #55958=>1838 param term

,term,tf,df
0,00,1649,1571
1,0000,1164,210
2,0000000000000000000000000000000000000000000000...,1465,1334
3,0000000000000000000000000000000000000000000000...,3534,3519
4,0000000000000000000000000000000000000000000000...,903,860
5,0000000000000000000000000000000000000000000000...,1465,1334
6,0000000000000000000000000000000000000000000000...,499,499
7,0000000000000000000000000000000000000000000000...,479,220
8,0000000000000000000000000000000000000000000000...,431,404
9,000000000046,638,135


after cleaning what kind of main api left

In [106]:
for api in api_li:
    print(corpus_df_[corpus_df_.term == api].values) # df<117 or tf<135 or df>19065 or tf>1834643

[['LoadLibrary' 307290 18734]]
[]
[['RegQueryValue' 1834643 19065]]
[]
[['CreateFile' 472341 18502]]
[]
[]
[['RegEnumValue' 190449 14362]]
[['RegCreateKey' 275635 13154]]
[['OpenProcess' 38665 1465]]
[['CreateRemoteThread' 19100 5396]]
[['CreateThread' 17568 4939]]
[['RegSetValue' 152672 9583]]
[['RegDeleteValue' 7129 2124]]
[['InternetOpen' 3063 1131]]
[['InternetConnect' 3081 1061]]
[['HttpSendRequest' 2793 1009]]
[['ExitProcess' 5195 5182]]
[['CreateProcess' 6786 4477]]
[['DeleteFile' 7422 4290]]
[['WinHttpOpen' 10282 206]]
[['WinHttpConnect' 10274 199]]
[['WinHttpOpenRequest' 10270 199]]
[['WinHttpSendRequest' 10300 198]]
[['WinHttpReadData' 9921 118]]
[['CopyFile' 90795 2308]]
[]
[['RegDeleteKey' 21305 816]]
[['TerminateProcess' 135 117]]
[['WinExec' 834 493]]
[]
[]


onehot: 太常見的用NIT、太少見的用UNK
* 找出超過得term有誰做成list，在這個list裡面的都轉換成NIT
* 找出低於的term有誰做成list，在這list裡面的或是都不再的轉成UNK
* one-hot

tf<135 / df<117  or tf>1834643 / df>19065 

In [150]:
upper_li = []
upper_li = corpus_df[corpus_df.tf > 1834643].term.tolist() #1
upper_li.extend(corpus_df[corpus_df.df > 19065].term.tolist()) #2
upper_li = list(set(upper_li)) #2
lower_li = []
lower_li = corpus_df[corpus_df.tf < 135].term.tolist() #52282
lower_li.extend(corpus_df[corpus_df.df < 117].term.tolist()) #53978
lower_li = list(set(lower_li)) #54118/52142
# for term in corpus_df[corpus_df.tf > 1834643].term.tolist()

In [162]:
special_token = pd.Series(['<NIT>','<UNK>','<PAD>'])
all_terms = corpus_df_['term'].append(special_token,ignore_index=True)

param_df = pd.get_dummies(all_terms)
param_df

,00,0000,000000000000000000000000000000000000000000000000000000000000,000000000000000000000000000000000000000000000000000000000000000000000000,0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000,0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000,0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000,00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000,000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000,000000000046,...,www,x,x86,xml,xpsp2res,y,yes,zip,zipfldr,zonemap
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [169]:
all_terms_li =  all_terms.tolist()
pickle.dump(obj=all_terms_li,file=open('data/all_terms_li_1841.pkl','wb'))
pickle.dump(obj=param_df,file=open('data/param_df_onehot_1841.pkl','wb'))
pickle.dump(obj=upper_li,file=open('data/upper_li_1841.pkl','wb'))
pickle.dump(obj=lower_li,file=open('data/lower_li_1841.pkl','wb'))

## length analysis

In [197]:
all_length=[]
for hkl in tqdm(next(os.walk('data/aries_simplified_profile_1002/'))[2]):
    with open('data/aries_simplified_profile_1002/' + hkl , encoding='ISO 8859-1') as f:
        lines = f.read().splitlines()
    for line in lines:
        temp = re.sub(dil," ",line)
        temp = temp.replace('PR','')
        temp = temp.replace('Ret','')
        temp = temp.replace('SUBK','')
        temp = temp.split(" ")
        temp = [x for x in temp if x]
        all_length.append(len(temp))



  0%|          | 0/19987 [00:00<?, ?it/s]

  0%|          | 53/19987 [00:00<00:38, 519.60it/s]

  0%|          | 94/19987 [00:00<00:43, 456.28it/s]

  1%|          | 135/19987 [00:00<00:46, 428.93it/s]

  1%|          | 182/19987 [00:00<00:45, 438.47it/s]

  1%|▏         | 267/19987 [00:00<00:38, 515.18it/s]

  2%|▏         | 333/19987 [00:00<00:36, 537.82it/s]

  2%|▏         | 418/19987 [00:00<00:33, 581.78it/s]

  3%|▎         | 500/19987 [00:00<00:31, 610.48it/s]

  3%|▎         | 569/19987 [00:00<00:31, 610.93it/s]

  3%|▎         | 648/19987 [00:01<00:30, 625.15it/s]

  4%|▎         | 722/19987 [00:01<00:30, 630.78it/s]

  4%|▍         | 792/19987 [00:01<00:30, 635.65it/s]

  4%|▍         | 870/19987 [00:01<00:29, 644.53it/s]

  5%|▍         | 950/19987 [00:01<00:29, 654.49it/s]

  5%|▌         | 1047/19987 [00:01<00:28, 674.67it/s]

  6%|▌         | 1133/19987 [00:01<00:27, 685.82it/s]

  6%|▌         | 1236/19987 [00:01<00:26, 705.19it/s]

  7%|▋         | 1331/19987 [00:01<0

In [204]:
# length analysis
all_length = np.array(all_length)
print(np.mean(all_length) , np.std(all_length) , np.min(all_length) , np.max(all_length))
print(np.quantile(all_length,0.25) , np.quantile(all_length,0.5) , np.quantile(all_length,0.75))
np.mean(all_length)+1*np.std(all_length) , np.quantile(all_length,0.75) +1.5*(np.quantile(all_length,0.75)-np.quantile(all_length,0.25))

8.185459594614215 2.7183130631315273 1 103
7.0 7.0 9.0


(10.903772657745742, 12.0)

## convert file to one-hot
1. 把檔案讀近來，若在upper_li轉換為NTT的one-hot
2. 若在param_df裡面all_term轉換為one-hot，若不再則轉換為UNK
3. 存成pickle

In [212]:
# kk=[]
# kk.append(param_df['<NIT>'].values)
# kk = np.array(kk)
kk[:11,:].shape

(11, 1841)

In [ ]:
dil= r"[,.;\-+^()/@#?!&$:{}\\*%~\'\"]+\ *" #等號、底線被保留

In [228]:
# single thread
pad_len = 11
for hkl in tqdm(next(os.walk('data/aries_simplified_profile_1002/'))[2]):
    with open('data/aries_simplified_profile_1002/' + hkl , encoding='ISO 8859-1') as f:
        lines = f.read().splitlines()
    onehot_all=np.empty((0,11,1841))
    for line in lines:
        onehot=[]
        temp = re.sub(dil," ",line)
        temp = temp.replace('PR','')
        temp = temp.replace('Ret','')
        temp = temp.replace('SUBK','')
        temp = temp.split(" ")
        temp = [x for x in temp if x]
        for term in temp:
            if term in upper_li:
                onehot.append(param_df['<NIT>'].values)
            elif term in all_terms_li:
                onehot.append(param_df[term].values)
            else:
                onehot.append(param_df['<UNK>'].values)
        if len(onehot) >= pad_len:
            onehot = np.array(onehot)
            onehot = onehot[:pad_len,:]
        else:
            iter_ = pad_len - len(onehot)
            for _ in range(iter_):
                onehot.append(param_df['<PAD>'].values)
            onehot = np.array(onehot)
        onehot = np.expand_dims(onehot,axis=0)
        onehot_all = np.append(onehot_all,onehot,axis=0)
        pickle.dump(obj=onehot_all,file=open('output/aries_simplified_profile_1002_param/'+hkl+'.pkl','wb'))
#         onehot_all = np.array(onehot_all)
#         print(onehot_all.shape)
#         break
#         onehot = np.expand_dims(onehot,axis=0)






  0%|          | 0/19987 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [234]:
args = []
for hkl in next(os.walk('data/aries_simplified_profile_1002/'))[2]: #input dir
    args.append(hkl)
def paramTransfer(args):
    hkl = args
    pad_len = 11
    with open('data/aries_simplified_profile_1002/' + hkl , encoding='ISO 8859-1') as f:
        lines = f.read().splitlines()
    onehot_all=np.empty((0,pad_len,1841))
    for line in lines:
        onehot=[]
        temp = re.sub(dil," ",line)
        temp = temp.replace('PR','')
        temp = temp.replace('Ret','')
        temp = temp.replace('SUBK','')
        temp = temp.split(" ")
        temp = [x for x in temp if x]
        for term in temp:
            if term in upper_li:
                onehot.append(param_df['<NIT>'].values)
            elif term in all_terms_li:
                onehot.append(param_df[term].values)
            else:
                onehot.append(param_df['<UNK>'].values)
        if len(onehot) >= pad_len:
            onehot = np.array(onehot)
            onehot = onehot[:pad_len,:]
        else:
            iter_ = pad_len - len(onehot)
            for _ in range(iter_):
                onehot.append(param_df['<PAD>'].values)
            onehot = np.array(onehot)
        onehot = np.expand_dims(onehot,axis=0)
        onehot_all = np.append(onehot_all,onehot,axis=0)
        pickle.dump(obj=onehot_all,file=open('output/aries_simplified_profile_1002_param/'+hkl+'.pkl','wb'))
#         print(hkl,'done')

In [235]:
wrong=[]
def handle_error(e):
    '''處理 child process 的錯誤'''
    traceback.print_exception(type(e), e, e.__traceback__)
    wrong.append((type(e), e, e.__traceback__))

In [ ]:
with Pool(processes=cpu_count()) as p: #CPU核心數目，決定提升速度的倍率
    print('Parent process {}.'.format(os.getpid()))  #爸爸pid，如果跑到抓狂要去linux砍掉的話
    p.map_async(paramTransfer,args,error_callback=handle_error)  #東西多的時候資源調度處理較好
    print('=============Joining all processes plz wait....=================')
    p.close()
    p.join()


Parent process 41535.
=============Joining all processes plz wait....=================


get lower and upper bound

In [107]:
for api in api_li:
    print(corpus_df[corpus_df.term == api].values) # df<117 or tf<135 or df>19065 or tf>1834643

[['LoadLibrary' 307290 18734]]
[]
[['RegQueryValue' 1834643 19065]]
[]
[['CreateFile' 472341 18502]]
[]
[['RegOpenCurrentUser' 1 1]]
[['RegEnumValue' 190449 14362]]
[['RegCreateKey' 275635 13154]]
[['OpenProcess' 38665 1465]]
[['CreateRemoteThread' 19100 5396]]
[['CreateThread' 17568 4939]]
[['RegSetValue' 152672 9583]]
[['RegDeleteValue' 7129 2124]]
[['InternetOpen' 3063 1131]]
[['InternetConnect' 3081 1061]]
[['HttpSendRequest' 2793 1009]]
[['ExitProcess' 5195 5182]]
[['CreateProcess' 6786 4477]]
[['DeleteFile' 7422 4290]]
[['WinHttpOpen' 10282 206]]
[['WinHttpConnect' 10274 199]]
[['WinHttpOpenRequest' 10270 199]]
[['WinHttpSendRequest' 10300 198]]
[['WinHttpReadData' 9921 118]]
[['CopyFile' 90795 2308]]
[]
[['RegDeleteKey' 21305 816]]
[['TerminateProcess' 135 117]]
[['WinExec' 834 493]]
[['GetUrlCacheEntryInfo' 58 32]]
[['WinHttpWriteData' 6 3]]


In [87]:
api_li = ['LoadLibrary'
 'RegOpenKey',
 'RegQueryValue',
 'RegCloseKey',
 'CreateFile',
 'CloseHandle',
 'RegOpenCurrentUser',
 'RegEnumValue',
 'RegCreateKey',
 'OpenProcess',
 'CreateRemoteThread',
 'CreateThread',
 'RegSetValue',
 'RegDeleteValue',
 'InternetOpen',
 'InternetConnect',
 'HttpSendRequest',
 'ExitProcess',
 'CreateProcess',
 'DeleteFile',
 'WinHttpOpen',
 'WinHttpConnect',
 'WinHttpOpenRequest',
 'WinHttpSendRequest',
 'WinHttpReadData',
 'CopyFile',
 'OpenThread',
 'RegDeleteKey',
 'TerminateProcess',
 'WinExec',
 'GetUrlCacheEntryInfo',
 'WinHttpWriteData']
len(api_li)

32

In [99]:
corpus_df

,term,tf,df
0,,3,3
1,,25,25
2,,1,1
3,15A,7,7
4,5G8205B,7,7
5,,2,1
6,g,3,3
7,,3,3
8,,8,5
9,,2,2


In [101]:
pickle.dump(file=open('output/asp1_tf_df.pkl','wb'),obj=corpus_df)

In [77]:
print(len(corpus)) #54032->56055->55958
sorted(corpus,reverse=True)

55958


['¢F',
 '¢D\x12',
 '¡Ó',
 '¡±',
 '¡Pinstall',
 'zzssbjslnbxhxhzb',
 'zzrljehbzlqrzevq',
 'zznkkrjxtsbrcvws',
 'zzjltzbxrrcscebl',
 'zzeznhjvzhncksek',
 'zzevqsenqbketsnl',
 'zzeknkvrjnseqxtz',
 'zz7',
 'zy',
 'zxzszhkscenjbbre',
 'zxqrjjhhrbbjlhlr',
 'zxnvqthhvtvvrebj',
 'zxnd',
 'zxlvhrbqrtnbxkhk',
 'zxcvcxzbds',
 'zxcvb',
 'zxbswnekbewnjjsv',
 'zwtwcvwbtwnkcetj',
 'zwtkcwnjjtqzrzbe',
 'zwrktlwvtkhwrwkj',
 'zwljklrslkvskkee',
 'zwjckrreeetkktkt',
 'zwhwhnqnqhqxvtst',
 'zwebnds',
 'zvtjvm',
 'zvpebfbsg',
 'zvelskzjjjhlbvbn',
 'zvarfjrrcre',
 'zu0y',
 'ztvunrar3',
 'ztvcabinet',
 'ztehrnrcjqllekzt',
 'ztdvl',
 'zsxrrktwsrlbnwqh',
 'zsrqekeezrnehhss',
 'zsqrbxthllljlehj',
 'zsnzlhtsvbsjjslj',
 'zsljnchrvhjhweqr',
 'zslbblqccsnvqlbj',
 'zskbnsbnehszsrhn',
 'zscchbkjjwqzrecz',
 'zsbtklrevzhzztjt',
 'zrtsnsnklkhrnjbt',
 'zrtqekwrezkhenst',
 'zrtkhshennbljklt',
 'zrtjvtjtnwrzjlwn',
 'zrthkweeljqbeqbe',
 'zrqvn',
 'zrnswrwnjtrzbrwj',
 'zrkxbwznlvjejnhj',
 'zrkezejslevxhbbq',
 'zrffratre',
 'z